# Target of this note

#### Mergre all datasets in one dataset and clean data

__In this Note i will do :__

* [About Dataset](#section1)<br>
    - * [read data set](#section2)<br/>
    - * [Merge data](#section2)<br/>
    - * [clean data](#section3)<br/>
    - * [save data](#section4)<br/>


<a id=section1></a>

# About Dataset

# Overview
This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks.
Not all users receive the same offer, and that is the challenge to solve with this data set.
Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.
Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.
You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer.

# Datasets
**portfolio.csv**

    • id (string) - offer id
    • offer_type (string) - type of offer ie BOGO, discount, informational
    • difficulty (int) - minimum required spend to complete an offer
    • reward (int) - reward given for completing an offer
    • duration (int) - time for offer to be open, in days
    • channels (list of strings)
**profile.csv**

    • age (int) - age of the customer
    • became_member_on (int) - date when customer created an app account
    • gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
    • id (str) - customer id
    • income (float) - customer's income
**transcript.csv**

    • event (str) - record description (ie transaction, offer received, offer viewed, etc.)
    • person (str) - customer id
    • time (int) - time in hours since start of test. The data begins at time t=0
    • value - (dict of strings) - either an offer id or transaction amount depending on the record

<a id=section1></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline


In [2]:
pd.set_option("display.max_columns",None)

# **Read Data**

In [3]:
portfolio=pd.read_csv("portfolio.csv")
profile=pd.read_csv("profile.csv")
transcript=pd.read_csv("transcript.csv")

In [4]:
portfolio.sample(10)

,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,id,channels/3
7,0,email,mobile,social,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,NaN
1,10,web,email,mobile,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,social
2,0,web,email,mobile,0,4,informational,3f207df678b143eea3cee63160fa8bed,NaN
6,2,web,email,mobile,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,social
9,2,web,email,mobile,10,7,discount,2906b810c7d4411798c6938adc9daaa5,NaN
5,3,web,email,mobile,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,social
3,5,web,email,mobile,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
0,10,email,mobile,social,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,NaN
4,5,web,email,NaN,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
8,5,web,email,mobile,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,social


In [5]:
profile.sample(10)

,gender,age,id,became_member_on,income
8778,M,75,25adc7b5ab704fcc9f8777ed914234ff,20170809,92000.0
6653,F,56,7d1dac0eb66147b89636f9e4aacb3d74,20180226,35000.0
5370,F,36,da37610be80246f69cb4792b7ca4ec25,20140404,31000.0
8632,M,55,97b8f814bb614ae28db8ea8adf4068ff,20160607,75000.0
5275,F,65,1d324572a0874ef480cc121408eb72d0,20171205,87000.0
2213,F,60,2d7e19d6760d4f94b6a84ddd487d5dac,20180410,52000.0
10086,F,53,0975267d50f7470da79d18591352e13d,20171207,62000.0
7260,NaN,118,8490563e978e4707b584e35ad1d56125,20171117,NaN
12691,F,62,f86af496c16140a5a9e4fbe605a27bb5,20170828,97000.0
84,NaN,118,a89752cdfdf24779aa22faa2bc24e7c2,20150920,NaN


In [6]:
transcript.sample(10)

,person,event,value/offer id,time,value/amount,value/offer_id,value/reward
2602,513e6bbe878c4b199c1d813fd54461a4,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN
7240,3423953f128c4cdc84b123eb6f5f4c40,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN
34736,bf4daf7b9b204d9f9b5e65a3c994009a,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN
46193,3b163517605a48a6a80dcf59cbad3b6f,offer viewed,ae264e3637204a6fb9bb56bc8210ddfd,36,NaN,NaN,NaN
17217,46c83dc0af064389b29ccf97d9c7ed57,offer completed,NaN,6,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0
42445,800a7b1aa057418f9d8e35bbc50a7880,offer viewed,5a8bc65990b245e5a138643cd4eb9837,24,NaN,NaN,NaN
18752,44960a76fc1f4626b9c55f8b4acd39c4,offer completed,NaN,12,NaN,ae264e3637204a6fb9bb56bc8210ddfd,10.0
13072,153e3e2d8ca342eeb5bcdbaeb7552568,offer viewed,3f207df678b143eea3cee63160fa8bed,0,NaN,NaN,NaN
43119,eae30c9121964155af5d7d08bdb9274b,transaction,NaN,24,4.04,NaN,NaN
53854,6b91411fca31438dbb986a6056291bbd,transaction,NaN,60,2.94,NaN,NaN


In [7]:
portfolio.shape

(10, 9)

In [8]:
profile.shape

(17000, 5)

In [9]:
transcript.shape

(59980, 7)

<a id=section2></a>

# Merge

- After mergre i want to understand each datasets

In [10]:
transcript.person.value_counts()

a95c94e9e8444d3ea4e10d2b96c6bba4    15
26de843918834f53b73844559b4b90f2    15
e67ada5fe1e240879e7a104257b196f8    15
f3cd5ab3f77e491a91d854e3f04d04c3    15
e4087b3f387b47e38d58e920f779056b    15
                                    ..
c4556c7bb4ab4c9d8e5fbf15c450b2b8     1
de2fdfba151e4fa6a940412e771316c4     1
537d1682c42740218162e3bca53b9552     1
49589dc19aec449f813a3486fc004af3     1
6af556ad6e3a4046b8eec8156295d9e9     1
Name: person, Length: 14173, dtype: int64

In [11]:
transcript["value/offer id"].value_counts()

fafdcd668e3743c1bb461111dcafc2a4    4555
f19421c1d4aa40978ebb69ca19b0e20d    4391
2298d6c36e964ae4a3e7e9706d1fb8c2    4229
4d5c57ea9a6940dd891ad53e9dbe8da0    4184
ae264e3637204a6fb9bb56bc8210ddfd    4154
5a8bc65990b245e5a138643cd4eb9837    4086
3f207df678b143eea3cee63160fa8bed    3454
2906b810c7d4411798c6938adc9daaa5    3395
9b98b8c7a33c4b65b9aebfe6a799e6d9    3338
0b1e1539f2cc45b7b9fa7c272da2e1d7    3095
Name: value/offer id, dtype: int64

**To merge transcript and portfolio we need to one column common  with two , so that i will rename name of id column in portfolio to value/offer id like in  transcript dataset**

In [12]:
portfolio=portfolio.rename(columns={"id":"value/offer id"})

In [13]:
portfolio

,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,value/offer id,channels/3
0,10,email,mobile,social,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,NaN
1,10,web,email,mobile,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,social
2,0,web,email,mobile,0,4,informational,3f207df678b143eea3cee63160fa8bed,NaN
3,5,web,email,mobile,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
4,5,web,email,NaN,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
5,3,web,email,mobile,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,social
6,2,web,email,mobile,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,social
7,0,email,mobile,social,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,NaN
8,5,web,email,mobile,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,social
9,2,web,email,mobile,10,7,discount,2906b810c7d4411798c6938adc9daaa5,NaN


In [14]:
#mergre between transcript and portfolio
df1=transcript.merge(portfolio,how="outer",on="value/offer id")

In [15]:
df1

,person,event,value/offer id,time,value/amount,value/offer_id,value/reward,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,channels/3
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
1,ebe7ef46ea6f4963a7dd49f501b26779,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
2,f082d80f0aac47a99173ba8ef8fc1909,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
3,c0d210398dee4a0895b24444a5fcd1d2,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
4,57dd18ec5ddc46828afb81ec5977bef2,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59975,b311fa5c3dab428abc630f466e74302c,transaction,NaN,90,15.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59976,325a76dbd5de4f62b1230ea9659f26fa,transaction,NaN,90,33.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59977,33e4e9352ea04b559e0d0a210642ecb3,transaction,NaN,90,8.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59978,059de3bbf78b497fa190ecbc20d26521,transaction,NaN,90,10.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**To merge df1 and profile we need to one column common  with two , so that i will rename name of id column in profile to person like in  df1 dataset**

In [16]:
profile=profile.rename(columns={"id":"person"})


In [17]:
profile

,gender,age,person,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [18]:
#mergre between all datasets
data=profile.merge(df1,how="inner",on="person")

In [19]:
data

,gender,age,person,became_member_on,income,event,value/offer id,time,value/amount,value/offer_id,value/reward,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,channels/3
0,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,transaction,NaN,18,21.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
2,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
4,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59975,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,offer received,2298d6c36e964ae4a3e7e9706d1fb8c2,0,NaN,NaN,NaN,3.0,web,email,mobile,7.0,7.0,discount,social
59976,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,offer viewed,2298d6c36e964ae4a3e7e9706d1fb8c2,6,NaN,NaN,NaN,3.0,web,email,mobile,7.0,7.0,discount,social
59977,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,transaction,NaN,54,21.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59978,F,62,e4052622e5ba45a8b96b59aba68cf068,20170722,82000.0,offer completed,NaN,54,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


----------------------------

<a id=section3></a>

# Clean Data

In [20]:
df=data.copy()

In [21]:
df.shape

(59980, 19)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59980 entries, 0 to 59979
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            52994 non-null  object 
 1   age               59980 non-null  int64  
 2   person            59980 non-null  object 
 3   became_member_on  59980 non-null  int64  
 4   income            52994 non-null  float64
 5   event             59980 non-null  object 
 6   value/offer id    38881 non-null  object 
 7   time              59980 non-null  int64  
 8   value/amount      16923 non-null  float64
 9   value/offer_id    4176 non-null   object 
 10  value/reward      4176 non-null   float64
 11  reward            38881 non-null  float64
 12  channels/0        38881 non-null  object 
 13  channels/1        38881 non-null  object 
 14  channels/2        35786 non-null  object 
 15  difficulty        38881 non-null  float64
 16  duration          38881 non-null  float6

__In this process i will do :__

1) Convert became_member_on to datetime

2) drop same features is'nt important

3) check missing values

4) check duplicted  



> **1) Convert became_member_on to datetime**

In [23]:
df.became_member_on

0        20170715
1        20170509
2        20170509
3        20170509
4        20170509
           ...   
59975    20170722
59976    20170722
59977    20170722
59978    20170722
59979    20170722
Name: became_member_on, Length: 59980, dtype: int64

In [24]:
#first convert to string to easy to insert - 
df.became_member=df.became_member_on.astype("O")

#second insert - between year , month and day
def date(x):
    x=str(x)
    return x[:4]+"-"+x[4:6]+"-"+x[-2:]

#second insert - between year , month and day
df.became_member=df.became_member.apply(date)

df.became_member

/tmp/ipykernel_6276/1388226675.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.became_member=df.became_member_on.astype("O")


0        2017-07-15
1        2017-05-09
2        2017-05-09
3        2017-05-09
4        2017-05-09
            ...    
59975    2017-07-22
59976    2017-07-22
59977    2017-07-22
59978    2017-07-22
59979    2017-07-22
Name: became_member_on, Length: 59980, dtype: object

In [25]:
#thrid convert to datetime
df.became_member=pd.to_datetime(df.became_member)


In [26]:
df.became_member

0       2017-07-15
1       2017-05-09
2       2017-05-09
3       2017-05-09
4       2017-05-09
           ...    
59975   2017-07-22
59976   2017-07-22
59977   2017-07-22
59978   2017-07-22
59979   2017-07-22
Name: became_member_on, Length: 59980, dtype: datetime64[ns]

In [27]:
df.columns

Index(['gender', 'age', 'person', 'became_member_on', 'income', 'event',
       'value/offer id', 'time', 'value/amount', 'value/offer_id',
       'value/reward', 'reward', 'channels/0', 'channels/1', 'channels/2',
       'difficulty', 'duration', 'offer_type', 'channels/3'],
      dtype='object')

In [28]:
#extract new features from became_member

df.insert(loc=4,value=df.became_member.dt.year,column="became_member_year") # year
df.insert(loc=5,value=df.became_member.dt.month,column="became_member_month") # month
df.insert(loc=6,value=df.became_member.dt.day,column="became_member_day") # day


In [29]:
df.head()

,gender,age,person,became_member_on,became_member_year,became_member_month,became_member_day,income,event,value/offer id,time,value/amount,value/offer_id,value/reward,reward,channels/0,channels/1,channels/2,difficulty,duration,offer_type,channels/3
0,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,2017,7,15,112000.0,transaction,NaN,18,21.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
2,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN
4,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,2017,5,9,100000.0,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,NaN,5.0,web,email,mobile,5.0,7.0,bogo,NaN


In [30]:
min_day=df.became_member.min()
min_day

Timestamp('2013-07-29 00:00:00')

In [31]:
max_day=df.became_member.max()
max_day

Timestamp('2018-07-26 00:00:00')

In [32]:
d

NameError: name 'd' is not defined

In [ ]:
# extract new feature to show how long customer download the application 

max_day=df.became_member.max()

value=(max_day-df.became_member).dt.days
df.insert(loc=7,value=value,column="login_days") # login_days



In [ ]:
# now we didnt need became_member_on 
df.drop("became_member_on",axis=1,inplace=True)

In [ ]:
df.head()

-------------------------

> **2) drop same features is'nt important**

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.drop(['value/amount', 'value/offer_id', 'value/reward','channels/0', 'channels/1', 'channels/2','channels/3'],axis=1,inplace=True)

In [ ]:
df

--------------------------

> **3) check missing values**

In [ ]:
round(df.isna().sum()/df.shape[0],2)*100

In [ ]:
df.gender.value_counts()

In [ ]:
df[df.gender.isna()]

- note that all missing value in gender same missing values in income ,sothat i will drop thoes values

In [ ]:
df.dropna(subset=["gender"],inplace=True)

In [ ]:
#check
round(df.isna().sum()/df.shape[0],2)*100

In [ ]:
df[df["value/offer id"].isna()]

missing values in [value/offer id ,reward ,difficulty ,duration and offer_type] is represended to a customer doesn't have offer

i will put all missing values by consent value equal 0

In [ ]:
df.fillna(0,inplace=True)

In [ ]:
#check
round(df.isna().sum()/df.shape[0],2)*100

-----------------------

> **4) check duplicted**

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated()]

we didn't have duplicated rows

--------------------

<a id=section4></a>

# Save data

In [ ]:
df.to_csv("data",index=False)